In [31]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [47]:
def get_tmdb_recommendations(movie_title, api_key, top_n=10):
    try:
        search_url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}'
        search_response = requests.get(search_url)
        search_response.raise_for_status()
        search_data = search_response.json()

        if not search_data['results']:
            print(f"'{movie_title}' movie is not found in TMDb.")
            return None

        movie_id = search_data['results'][0]['id']

        recommendations_url = f'https://api.themoviedb.org/3/movie/{movie_id}/recommendations?api_key={api_key}&language=en-EN&page=1'
        recommendations_response = requests.get(recommendations_url)
        recommendations_response.raise_for_status()
        recommendations_data = recommendations_response.json()

        if not recommendations_data['results']:
            print(f"No recommedations for '{movie_title}' movie.")
            return None

        df = pd.DataFrame(recommendations_data['results'])
        df['overview'] = df['overview'].fillna('')

        tfidf = TfidfVectorizer(stop_words='english')  
        tfidf_matrix = tfidf.fit_transform(df['overview'])
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

        search_title_lower = movie_title.lower() 
        df_titles_lower = df['title'].str.lower() 
        idx = df.index[df_titles_lower == search_title_lower].tolist() 

        if not idx: 
            idx = 0 
        else:
            idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n + 1]
        movie_indices = [i[0] for i in sim_scores]

        return df['title'].iloc[movie_indices]

    except requests.exceptions.RequestException as e:
        print(f"Request error to TMDb API: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [49]:
def get_tmdb_overview(movie_title, api_key):
    search_url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}'
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        data = response.json()
        if data['results']:
            return data['results'][0].get('overview', '')
        else:
            return ''
    except requests.exceptions.RequestException as e:
        print(f'Error getting description from TMDB: {e}')
        return ''

In [51]:
api_key = "b8b61d90e28e284765d67d8d8937dc32" 
movie_title = "Deadpool"

recommendations = get_tmdb_recommendations(movie_title, api_key)
overview = get_tmdb_overview(movie_title, api_key)

if recommendations is not None:
    print(f"Recommendations for '{movie_title}':")
    print(recommendations)

if overview:
    print(f"\nDescription for '{movie_title}': {overview}")

Recommendations for 'Deadpool':
2                         Doctor Strange
4           Star Wars: The Force Awakens
15                    Mad Max: Fury Road
19                       Kung Fu Panda 3
12                               Ant-Man
1             Captain America: Civil War
3     Batman v Superman: Dawn of Justice
5                               Zootopia
6                Guardians of the Galaxy
7                           The Avengers
Name: title, dtype: object

Description for 'Deadpool': The origin story of former Special Forces operative turned mercenary Wade Wilson, who, after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilities and a dark, twisted sense of humor, Deadpool hunts down the man who nearly destroyed his life.
